In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.linear_model import Ridge

from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix

import sys
sys.path.append('../../modules')
from helper_functions import feature_reduction_x, rating_feature_conversion


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/jbyrneirl/BER-Rating-AI-Model/main/data/training/BERRatingData_aa.csv', sep=';', on_bad_lines="skip", low_memory=False)

np.random.seed(0)

#X = df.drop("BerRating", axis='columns')
#X = pd.get_dummies(X)
X = feature_reduction_x(df)

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X)
X = imp.transform(X)
y = df.BerRating


X_train, X_test, y_train, y_test = train_test_split( X, y, random_state=2)

c:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [4]:
model = Ridge()


model.fit(X_train , y_train)
score = model.score(X_test , y_test)

y_hat = model.predict(X_test)

y_con = rating_feature_conversion(y_test.to_numpy())

[134.25675421 234.76057808  61.19077712 ...  67.66359733 217.81094825
  83.28876661]


In [6]:
print("Score: ", score, "\n")

cm = confusion_matrix(y_con, rating_feature_conversion(y_hat))

print("CM")
print(cm)

Score:  0.8116193220720149 

CM
[[  4   3   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [ 35 109 107  17   1   0   0   0   0   0   0   0   0   0   0]
 [ 26 121 186  43  10   4   0   0   0   0   0   0   0   0   0]
 [  3   5  10  22  15  11   3   0   1   0   0   0   0   1   0]
 [  0   1   6  16  31  44  18   8   0   0   0   1   1   0   0]
 [  1   2   0   3  33  73  57  29  14   5   1   1   0   0   0]
 [  0   0   2   1  18  63 107  79  34  14   2   2   0   0   0]
 [  0   0   0   2   4  15  54  70  45  31  13   5   0   1   0]
 [  0   0   0   0   1   6  21  47  44  43  24   8   1   2   1]
 [  0   0   0   0   1   5   3  24  36  54  27  14   7   2   1]
 [  0   0   0   0   0   0   1  11  23  38  42  21  12   9   3]
 [  0   0   0   0   0   0   0   1   3  13  13  15  12  18   7]
 [  0   0   0   0   0   0   0   0   0   6  12   9  10  26  11]
 [  0   0   0   0   0   0   0   0   0   1   4   9  14  18  19]
 [  0   0   0   0   0   0   0   0   0   1   1   5   5  18  73]]
